# An Analysis of Form vs Fixtures

In [10]:
import numpy as np
import pandas as pd
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

## 1. Load and Merge Player Data with Team Data

In [11]:
# Load player data
def_df = pd.read_csv('def_finaldat.csv')
att_df = pd.read_csv('att_finaldat.csv')
final_df = pd.concat([def_df, att_df], axis=0, ignore_index=True)

print(f"final_df shape: {final_df.shape}")
print(f"Date range: {final_df['Date'].min()} to {final_df['Date'].max()}")

final_df shape: (62884, 137)
Date range: 2017-08-11 to 2024-05-19


In [12]:
# Load team data
team_df = pd.read_csv('team_finaldat.csv', index_col=0)

print(f"team_df shape: {team_df.shape}")
print(f"Team data columns: {team_df.columns.tolist()}")

team_df shape: (5320, 18)
Team data columns: ['Date', 'Time', 'Matchweek', 'Day', 'Venue', 'Result', 'Goals Scored', 'Goals Conceded', 'Opponent', 'xG', 'xGA', 'Possession', 'Attendance', 'Captain', 'Formation', 'Opposition Formation', 'Referee', 'Team']


In [13]:
# Merge final_df with team data
# We merge on Date and Team to add team-level context to each player's match performance
merged_df = final_df.merge(
    team_df,
    on=['Date', 'Team'],
    how='left',
    suffixes=('', '_team')
)

print(f"Merge complete!")
print(f"Original final_df: {final_df.shape}")
print(f"Merged dataset: {merged_df.shape}")
print(f"Merge success rate: {100 * merged_df['Formation'].notna().sum() / len(merged_df):.1f}%")

Merge complete!
Original final_df: (62884, 137)
Merged dataset: (62884, 153)
Merge success rate: 100.0%


In [14]:
# Clean up duplicate columns
# Drop team-suffixed columns that are identical to player data
columns_to_drop = ['Matchweek_team', 'Day_team', 'Venue_team', 'Opponent_team', 'Result_team']
merged_df = merged_df.drop(columns=columns_to_drop, errors='ignore')

print(f"After cleanup: {merged_df.shape}")
print(f"\nNew columns added from team data:")
new_cols = ['Time', 'Goals Scored', 'Goals Conceded', 'xG_team', 'xGA', 
            'Possession', 'Attendance', 'Captain', 'Formation', 
            'Opposition Formation', 'Referee']
for col in new_cols:
    if col in merged_df.columns:
        print(f"  ✓ {col}")

After cleanup: (62884, 148)

New columns added from team data:
  ✓ Time
  ✓ Goals Scored
  ✓ Goals Conceded
  ✓ xG_team
  ✓ xGA
  ✓ Possession
  ✓ Attendance
  ✓ Captain
  ✓ Formation
  ✓ Opposition Formation
  ✓ Referee


## 2. Add Fixture Difficulty Rating (FDR)

We'll add FDR using:
- **2017-18**: Predictions from calibrated ordinal regression model
- **2018-19 onwards**: Observed FPL FDR data

In [15]:
# Team name harmonization (FPL naming conventions)
team_name_harmonization = {
    # Core mappings
    'Manchester City': 'Man City',
    'Manchester Utd': 'Man Utd',
    'Tottenham': 'Spurs',
    'Newcastle Utd': 'Newcastle',
    'Leicester City': 'Leicester',
    
    # Promoted team fixes
    'Leeds United': 'Leeds',
    "Nott'ham Forest": "Nott'm Forest",
    'Nottingham': "Nott'm Forest",
    'Norwich City': 'Norwich',
    'Sheffield': 'Sheffield Utd',
    'Luton Town': 'Luton',
    
    # Other teams
    'Cardiff City': 'Cardiff',
    'Huddersfield Town': 'Huddersfield',
    'Swansea City': 'Swansea',
    'Stoke City': 'Stoke',
}

merged_df['Team'] = merged_df['Team'].replace(team_name_harmonization)
merged_df['Opponent'] = merged_df['Opponent'].replace(team_name_harmonization)

print("✓ Applied team name harmonization")

✓ Applied team name harmonization


In [16]:
# Load observed FPL FDR data (2018-19 onwards)
fpl_data_path = Path('Fantasy-Premier-League/data')
all_fixtures = []
all_teams_mapping = {}

for season_folder in sorted(fpl_data_path.iterdir()):
    if not season_folder.is_dir() or season_folder.name < '2018-19':
        continue

    fixtures_file = season_folder / 'fixtures.csv'
    if fixtures_file.exists():
        df = pd.read_csv(fixtures_file)
        df['Season'] = season_folder.name
        all_fixtures.append(df)

    teams_file = season_folder / 'teams.csv'
    if teams_file.exists():
        teams = pd.read_csv(teams_file)
        all_teams_mapping[season_folder.name] = dict(zip(teams['id'], teams['name']))
    else:
        raw_file = season_folder / 'raw.json'
        if raw_file.exists():
            with open(raw_file, 'r') as f:
                data = json.load(f)
                if 'teams' in data:
                    all_teams_mapping[season_folder.name] = {t['id']: t['name'] for t in data['teams']}

fixtures_df = pd.concat(all_fixtures, ignore_index=True)
print(f"✓ Loaded {len(fixtures_df)} FPL fixtures")

✓ Loaded 2660 FPL fixtures


In [17]:
# Create observed FDR lookup
observed_fdr = []
for _, fixture in fixtures_df.iterrows():
    season = fixture['Season']
    if season not in all_teams_mapping:
        continue

    team_h_name = all_teams_mapping[season].get(fixture['team_h'])
    team_a_name = all_teams_mapping[season].get(fixture['team_a'])

    if not team_h_name or not team_a_name:
        continue

    kickoff_time = pd.to_datetime(fixture.get('kickoff_time', None))
    if pd.isna(kickoff_time):
        continue
    kickoff_time = kickoff_time.tz_localize(None)

    if pd.notna(fixture['team_h_difficulty']):
        observed_fdr.append({
            'Date': kickoff_time,
            'Team': team_h_name,
            'Opponent': team_a_name,
            'Venue': 'Home',
            'FDR': int(fixture['team_h_difficulty'])
        })

    if pd.notna(fixture['team_a_difficulty']):
        observed_fdr.append({
            'Date': kickoff_time,
            'Team': team_a_name,
            'Opponent': team_h_name,
            'Venue': 'Away',
            'FDR': int(fixture['team_a_difficulty'])
        })

observed_fdr_df = pd.DataFrame(observed_fdr)
observed_fdr_df['Date'] = observed_fdr_df['Date'].dt.date
print(f"✓ Created observed FDR lookup: {len(observed_fdr_df)} records")

✓ Created observed FDR lookup: 5320 records


In [18]:
# Load 2017-18 predictions from FDR_Imputation_Model.ipynb
# The model training was done in that notebook - we just use the predictions here
pred_1718 = pd.read_csv('FDR /fixtures_2017-18_predicted.csv')

# Match predictions to actual match dates from team_finaldat
team_df_1718 = pd.read_csv('team_finaldat.csv', index_col=0)
team_df_1718['Date'] = pd.to_datetime(team_df_1718['Date'])

def get_season(date):
    if date.month >= 8:
        return f'{date.year}-{str(date.year+1)[-2:]}'
    else:
        return f'{date.year-1}-{str(date.year)[-2:]}'

team_df_1718['Season'] = team_df_1718['Date'].apply(get_season)
team_df_1718 = team_df_1718[team_df_1718['Season'] == '2017-18']
team_df_1718['Team'] = team_df_1718['Team'].replace(team_name_harmonization)
team_df_1718['Opponent'] = team_df_1718['Opponent'].replace(team_name_harmonization)

# Create 2017-18 FDR with actual dates
pred_1718_with_dates = []
for _, match in team_df_1718.iterrows():
    pred_match = pred_1718[
        (pred_1718['team_h'] == match['Team']) &
        (pred_1718['team_a'] == match['Opponent'])
    ]

    if len(pred_match) == 0:
        pred_match = pred_1718[
            (pred_1718['team_h'] == match['Opponent']) &
            (pred_1718['team_a'] == match['Team'])
        ]

        if len(pred_match) > 0:
            pred_1718_with_dates.append({
                'Date': match['Date'],
                'Team': match['Team'],
                'Opponent': match['Opponent'],
                'FDR': int(pred_match.iloc[0]['team_a_difficulty'])
            })
    else:
        pred_1718_with_dates.append({
            'Date': match['Date'],
            'Team': match['Team'],
            'Opponent': match['Opponent'],
            'FDR': int(pred_match.iloc[0]['team_h_difficulty'])
        })

pred_1718_df = pd.DataFrame(pred_1718_with_dates)
pred_1718_df['Date'] = pd.to_datetime(pred_1718_df['Date']).dt.date
print(f"✓ Created 2017-18 FDR lookup: {len(pred_1718_df)} records")

✓ Created 2017-18 FDR lookup: 760 records


In [19]:
# Combine all FDR data
all_fdr = pd.concat([pred_1718_df, observed_fdr_df], ignore_index=True)
all_fdr = all_fdr.drop_duplicates(subset=['Date', 'Team', 'Opponent'])
print(f"✓ Combined FDR lookup: {len(all_fdr)} records")

✓ Combined FDR lookup: 6080 records


In [20]:
# Merge FDR into merged_df
merged_df['Date'] = pd.to_datetime(merged_df['Date']).dt.date
merged_df = merged_df.merge(
    all_fdr[['Date', 'Team', 'Opponent', 'FDR']],
    on=['Date', 'Team', 'Opponent'],
    how='left'
)

print(f"✓ Merged FDR column")
print(f"  FDR coverage: {merged_df['FDR'].notna().sum() / len(merged_df) * 100:.1f}%")

✓ Merged FDR column
  FDR coverage: 100.0%


## 3. Validation: FDR Coverage by Season

In [21]:
# Add season column
merged_df['Season'] = merged_df['Date'].apply(get_season)

print("="*80)
print("FDR COVERAGE BY SEASON")
print("="*80)
for season in sorted(merged_df['Season'].unique()):
    season_data = merged_df[merged_df['Season'] == season]
    with_fdr = season_data['FDR'].notna().sum()
    total = len(season_data)
    pct = with_fdr / total * 100
    status = "✅" if pct >= 99 else "⚠️ " if pct > 90 else "❌"
    print(f"{season}: {with_fdr:>5,} / {total:>5,} ({pct:>5.1f}%) {status}")

print("\n" + "="*80)
print("PROMOTED TEAM VERIFICATION")
print("="*80)

for team in ['Norwich', 'Sheffield Utd', 'Luton', 'Leeds', "Nott'm Forest"]:
    team_data = merged_df[merged_df['Team'] == team]
    if len(team_data) > 0:
        with_fdr = team_data['FDR'].notna().sum()
        total = len(team_data)
        pct = with_fdr / total * 100
        status = "✅" if pct >= 99 else "⚠️ "
        print(f"{team:<20} {with_fdr:>5,} / {total:>5,} ({pct:>5.1f}%) {status}")

FDR COVERAGE BY SEASON
2017-18: 8,770 / 8,770 (100.0%) ✅
2018-19: 9,060 / 9,060 (100.0%) ✅
2019-20: 9,039 / 9,039 (100.0%) ✅
2020-21: 8,686 / 8,686 (100.0%) ✅
2021-22: 8,843 / 8,843 (100.0%) ✅
2022-23: 9,319 / 9,319 (100.0%) ✅
2023-24: 9,167 / 9,167 (100.0%) ✅

PROMOTED TEAM VERIFICATION
Norwich                820 /   820 (100.0%) ✅
Sheffield Utd        1,342 / 1,342 (100.0%) ✅
Luton                  431 /   431 (100.0%) ✅
Leeds                1,369 / 1,369 (100.0%) ✅
Nott'm Forest          971 /   971 (100.0%) ✅


## 4. Final Dataset Summary

The `merged_df` dataset now contains:
- **Player-level performance stats** (137 original columns)
- **Team-level match context** (10 columns from team_finaldat)
- **Fixture Difficulty Rating (FDR)** (1 new column)

**Total: 149 columns × 62,884 observations**

Ready for analysis!

In [22]:
print("="*80)
print("FINAL DATASET SUMMARY")
print("="*80)
print(f"Shape: {merged_df.shape}")
print(f"\nKey columns:")
print(f"  - Date, Team, Opponent, Venue, Result")
print(f"  - Position, Minutes Played, total_points")
print(f"  - Formation, Opposition Formation, Possession")
print(f"  - FDR (Fixture Difficulty Rating)")
print(f"\nFDR coverage: {merged_df['FDR'].notna().sum()} / {len(merged_df)} ({merged_df['FDR'].notna().sum() / len(merged_df) * 100:.1f}%)")
print(f"\nDate range: {merged_df['Date'].min()} to {merged_df['Date'].max()}")
print(f"Seasons: {', '.join(sorted(merged_df['Season'].unique()))}")

FINAL DATASET SUMMARY
Shape: (62884, 150)

Key columns:
  - Date, Team, Opponent, Venue, Result
  - Position, Minutes Played, total_points
  - Formation, Opposition Formation, Possession
  - FDR (Fixture Difficulty Rating)

FDR coverage: 62884 / 62884 (100.0%)

Date range: 2017-08-11 to 2024-05-19
Seasons: 2017-18, 2018-19, 2019-20, 2020-21, 2021-22, 2022-23, 2023-24


In [23]:
# Display sample with FDR
print("\nSample data with FDR:")
sample_cols = ['Date', 'Team', 'Opponent', 'Position', 'Minutes Played', 
               'Goals', 'xG', 'total_points', 'Formation', 'FDR']
merged_df[sample_cols].head(10)


Sample data with FDR:


,Date,Team,Opponent,Position,Minutes Played,Goals,xG,total_points,Formation,FDR
0,2023-08-12,Bournemouth,West Ham,RB,74,0,0.0,2,4-2-3-1,2
1,2023-08-19,Bournemouth,Liverpool,RB,90,0,0.0,1,4-2-3-1,5
2,2023-08-26,Bournemouth,Spurs,RB,90,0,0.0,1,4-2-3-1,3
3,2023-09-02,Bournemouth,Brentford,RB,90,0,0.0,0,4-2-3-1,2
4,2023-09-17,Bournemouth,Chelsea,RB,90,0,0.0,6,4-2-3-1,3
5,2023-09-24,Bournemouth,Brighton,RB,90,0,0.0,1,4-2-3-1,3
6,2023-09-30,Bournemouth,Arsenal,RB,90,0,0.0,0,4-2-3-1,5
7,2023-10-07,Bournemouth,Everton,RB,5,0,0.0,1,4-2-3-1,2
8,2023-10-21,Bournemouth,Wolves,RB,90,0,0.0,1,4-2-3-1,2
9,2023-10-28,Bournemouth,Burnley,RB,84,0,0.0,2,4-2-3-1,2
